# 6-dimensional CSLA - NB 00 - Start here

This notebook is adjointed to the article "The moduli space of left invariant metrics on six-dimensional characteristically solvable Lie algebras" by I. Cardoso, A. Cosgaya, and S. Reggiani. Here we will state the code for the basic Sagemath objects and definitions to perform the computations for the article, hence the need to *start here*.


## 1. Structure coefficients and construction of the Lie Algebras

We start by loading a Sagemaht object which contains a dictionary whose keys are the algebra names and whose values are the structure coefficient 6-list. This dictionary has already been defined hence there is no need to entry the data each time.

In [1]:
algebras_nums_dict = load('algebras_nums_dict')
algebras_nums_dict

{'h1': ['0', '0', '0', '0', '0', '0'],
 'h2': ['0', '0', '0', '0', '12', '34'],
 'h3': ['0', '0', '0', '0', '0', '12+34'],
 'h4': ['0', '0', '0', '0', '12', '14+23'],
 'h5': ['0', '0', '0', '0', '13+42', '14+23'],
 'h6': ['0', '0', '0', '0', '12', '13'],
 'h7': ['0', '0', '0', '12', '13', '23'],
 'h8': ['0', '0', '0', '0', '0', '12'],
 'h9': ['0', '0', '0', '0', '12', '14+25'],
 'h10': ['0', '0', '0', '12', '13', '14'],
 'h11': ['0', '0', '0', '12', '13', '14+23'],
 'h12': ['0', '0', '0', '12', '13', '24'],
 'h13': ['0', '0', '0', '12', '13+14', '24'],
 'h14': ['0', '0', '0', '12', '14', '13+42'],
 'h15': ['0', '0', '0', '12', '13+42', '14+23'],
 'h16': ['0', '0', '0', '12', '14', '24'],
 'h17': ['0', '0', '0', '0', '12', '15'],
 'h18': ['0', '0', '0', '12', '13', '14+35'],
 'h19+': ['0', '0', '0', '12', '23', '14+35'],
 'h19-': ['0', '0', '0', '12', '23', '14-35'],
 'h20': ['0', '0', '0', '0', '12', '15+34'],
 'h21': ['0', '0', '0', '12', '14', '15'],
 'h22': ['0', '0', '0', '12', '14

Next we see the code for the function that will take each list of structure coeficients and turn it into a Lie Algebra for Sagemath.

In [2]:
def coef_str(nums):
    '''Input: A list containing the strings of numbers according to
    the Salamon classification Output: A dictionary containing the
    structure coefficient as required by the LieAlgebra class

    '''
    coef_str_dict = dict()
    for k in range(6):
        st = nums[k].replace(" ", "")
        sg = -1 
        if len(st) == 2:
            coef_str_dict[("e"+st[0], "e"+st[1])] = {"e"+str(k+1) : sg }
        elif len(st) == 5:
            coef_str_dict[("e"+st[0], "e"+st[1])] = {"e"+str(k+1) : sg }
            if st[2] == "-":
                sg = 1
                coef_str_dict[("e"+st[3], "e"+st[4])] = {"e"+str(k+1) : sg }
            else:
                coef_str_dict[("e"+st[3], "e"+st[4])] = {"e"+str(k+1) : sg }
        if len(st) == 8:
            coef_str_dict[("e"+st[0], "e"+st[1])] = {"e"+str(k+1) : sg }
            coef_str_dict[("e"+st[3], "e"+st[4])] = {"e"+str(k+1) : sg }
            coef_str_dict[("e"+st[6], "e"+st[7])] = {"e"+str(k+1) : sg }
    return coef_str_dict

The function **construct_the_algebra** inputs a key from the dictionary. It creates various (global) variables related to the Lie Algebra, which are self-understood from the code itself.

In [3]:
def construct_the_algebra(algebra_name):    
    global alg_name
    alg_name = algebra_name
    global label
    label = algebras_nums_dict[alg_name]
    global ce
    ce = coef_str(label)
    global ce_dict
    ce_dict = { alg_name : ce}
    global alg
    alg.<e1,e2,e3,e4,e5,e6> = LieAlgebra(SR, ce, nilpotent=True )
    global basis
    basis = alg.basis().values()
    global der_basis
    der_basis= alg.derivations_basis()
    global der_dim
    der_dim= len(der_basis)
    global der_vars
    der_vars = [var("d"+str(i)) for i in range(der_dim)]
    global der_gen
    der_gen = sum(der_vars[i] * der_basis[i] for i in range(der_dim))
    global Center
    Center = alg.center()
    global Center_dim
    Center_dim = Center.dimension()
    global Derivated
    Derivated = alg.derived_subalgebra()
    global Derivated_dim
    Derivated_dim = Derivated.dimension()

## 2. The algebra of derivations

Among the global variables one finds a few related to the algebra of derivations.

The function **generic_derivations** which we define below will give us the expression for a generic derivation. It also detects which of them have a lower triangular expression, and for that we define a few more functions.

In [4]:
def is_square(M):
    return M.nrows() == M.ncols()
def is_lower_triang(M):
    a = 0
    if is_square(M) == True:
        n = M.ncols()
        for i in range(n):
            for j in range(i+1,n):
                if M[i,j] != 0:
                    a = a+1
        return a == 0
    else:
        print('The matrix is not square.')
def is_s_lower_triang(M):
    a = 0
    if is_square(M) == True:
        n = M.ncols()
        for i in range(n):
            for j in range(i,n):
                if M[i,j] != 0:
                    a = a+1
        return a == 0
    else:
        print('The matrix is not square.')
def is_upper_triang(M):
    a = 0
    if is_square(M) == True:
        n = M.ncols()
        for i in range(n):
            for j in range(i):
                if M[i,j] != 0:
                    a = a+1
        return a == 0
    else:
        print('The matrix is not square.')
def is_diagonal(M):
    a = False
    if is_lower_triang(M) == True:
        if is_upper_triang(M) == True:
            a = True
    return a

In [5]:
def generic_derivations(algebra_name):
    construct_the_algebra(algebra_name)
    global der_basis_diag
    der_basis_diag = []
    global index_der_basis_diag
    index_der_basis_diag = []
    global der_basis_slt
    der_basis_slt = []
    global index_der_basis_slt
    index_der_basis_slt = []
    for i in range(der_dim):
            deri = der_basis[i]
            if is_diagonal(deri) == True:
                der_basis_diag.append(deri)
                index_der_basis_diag.append(i)
            if is_s_lower_triang(deri) == True:
                der_basis_slt.append(deri)
                index_der_basis_slt.append(i)
    global der_gen_diag
    der_gen_diag = diagonal_matrix(der_gen.diagonal()) #generic diagonal derivation
    global der_gen_slt
    der_gen_slt = der_gen - der_gen_diag #generic strictly triangular derivation
    global der_vars_diag
    der_vars_diag = [der_vars[i] for i in index_der_basis_diag]
    global der_vars_slt
    der_vars_slt = [der_vars[i] for i in index_der_basis_slt]

By aid of this function we are able to find which of the Lie Algebras of the list are CLSA. Moreover, each generic matrix is represented by a lower triangular matrix. This result is stated in Proposition 2.1 and a separate notebook will be given to show this result.

The next portion of code will update the dictionary to the notation for the structure coefficients obtained after performing the suitable change of basis for the algebras $\mathfrak{h}_9$, $\mathfrak{h}_{18}$, $\mathfrak{h}_{19}^+$ and $\mathfrak{h}_{26}^-$, according to Proposition 2.1. 

In [6]:
algebras_nums_dict_bis = { 'h9_bis' : [ '0', '0' , '0', '0', '21', '15+23'],
                           'h9_tris' :  [ '0' , '0' , '0' , '0' , '12', '51+23' ],
                           'h18_bis' : [ '0', '0' , '0', '12', '13', '15+24'],
                           'h19+_bis' : [ '0', '0' , '0', '32', '12', '14+35'],
                           'h19+_tris' : [ '0', '0' , '0' , '23' , '21', '14+35'],
                           'h26-_bis' : [ '0', '0' , '12', '31', '32', '15+24']}

algebras_nums_dict.update(algebras_nums_dict_bis)

A list of the names of the CSLA is also already stored in an object. We load it and make some useful new variables.

In [7]:
names_lower = load('Names_Lower_Algs_of_der_list.sobj')
names_bis = ['h9_bis','h18_bis','h19+_bis','h26-_bis']
names_tris = ['h9_bis','h9_tris', 'h18_bis','h19+_bis', 'h19+_tris', 'h26-_bis']
names = names_lower + names_tris
algebras_nums_l = {key: algebras_nums_dict[key] for key in names}

## 3. The generic automorphisms for CLSA

Next we share the code that constructs the generic automorphism for a CLSA. Please note that this algorithm strongly uses that the generic derivations have a lower triangular matrix form.

In [8]:
def first_appearence(matriz, variables):
    coefficient_dict = {}
    for d in variables:
        coefficient_dict[d] = sorted(matriz.find(lambda coef: coef == d,indices=True))[0]
    return coefficient_dict

def generic_automorphisms(algebra_name):
#    construct_the_algebra(algebra_name)
    generic_derivations(algebra_name)
    global auto_gen
    auto_gen = exp(der_gen)
    global aut_gen_slt
    aut_gen_slt = exp(der_gen_slt)
    global aut_gen_diag
    aut_gen_diag = exp(der_gen_diag)
    global aut_vars
    aut_vars =  [var("a"+str(i)) for i in range(der_dim)]
    global aut_vars_diag
    aut_vars_diag = [aut_vars[i] for i in index_der_basis_diag]
    global aut_vars_slt
    aut_vars_slt = [aut_vars[i] for i in index_der_basis_slt]
    global der_vars_coeffs
    der_vars_coeffs = first_appearence(der_gen,der_vars)
    A = aut_gen_slt
    for k in range(len(der_vars_slt)):
        d = der_vars_slt[k]
        a = aut_vars_slt[k]
        i,j = der_vars_coeffs[d]
        A = A.subs(d==a-A[i,j]+d)
    global aut_gen_diag_list 
    aut_gen_diag_list = [exp(der_vars_diag[k] * der_basis_diag[k]).subs(der_vars_diag[k]==log(aut_vars_diag[k])) for k in range(len(der_vars_diag))]
    for A_diag in aut_gen_diag_list:
        A = A * A_diag
        for k in range(len(aut_vars_slt)):
            d = der_vars_slt[k]
            a = aut_vars_slt[k]
            i,j = der_vars_coeffs[d]
            A = A.subs(a==a/(A[i,j]/a))
    A.simplify_full()
    global aut_gen
    aut_gen = A.simplify_full()

## 4. The generic metrics for CSLA

Finally the next function will construct the matrix for a generic metric. Again, this algorithm relies on the basis expression for the derivations and the automorphisms.

In [9]:
def generic_metric(algebra_name):
#    construct_the_algebra(algebra_name)
#    generic_derivations(algebra_name)
    generic_automorphisms(algebra_name)
    global aut_vars_coeffs
    aut_vars_coeffs = first_appearence(aut_gen,aut_vars)
    global Sigma
    Sigma = identity_matrix(SR, 6)
    aut_vars_coeffs_values = aut_vars_coeffs.values()
    count = 0
    for i in range(6):
        for j in range(i+1):
            if (i,j) not in aut_vars_coeffs_values:
                Sigma[i,j]=var("s"+str(count))
                count += 1
    global g
    g = transpose(Sigma)*Sigma


Finally, we point out that all of the above code is contained in the file "CSLA.sage", which is loaded in several of the notebooks for the article.

---

This notebook corresponds to the article "The moduli space of left-invariant metrics on six-dimensional characteristically solvable nilmanifolds" by I. Cardoso, A. Cosgaya, and S. Reggiani (2024).